In [ ]:
%%capture
# Install Unsloth and dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

print("Unsloth installed successfully!")

Upload Training Data

In [ ]:
print("Upload training files:")
print("1. sister_speech_recognition.json")
print("2. sister_conversation.json")

from google.colab import files
import json
import os

# Upload training files
uploaded = files.upload()

# Verify uploads
for filename in uploaded.keys():
    print(f"✅ Uploaded: {filename} ({len(uploaded[filename])} bytes)")

Upload training files:
1. sister_speech_recognition.json
2. sister_conversation.json


Saving sister_conversation.json to sister_conversation.json
Saving sister_speech_recognition.json to sister_speech_recognition.json
✅ Uploaded: sister_conversation.json (79614 bytes)
✅ Uploaded: sister_speech_recognition.json (116055 bytes)


Load and Prepare Data

In [ ]:
import json
import torch
from datasets import Dataset

def load_training_data():
    """Load sister's training data"""
    print(" Loading sister's training data...")

    # Load datasets
    with open('sister_speech_recognition.json', 'r', encoding='utf-8') as f:
        speech_data = json.load(f)

    with open('sister_conversation.json', 'r', encoding='utf-8') as f:
        conversation_data = json.load(f)

    print(f" Speech samples: {len(speech_data)}")
    print(f" Conversation samples: {len(conversation_data)}")

    return speech_data, conversation_data

def format_for_training(speech_data, conversation_data):
    """Format data for Gemma 3n training"""
    print(" Formatting data for training...")

    training_texts = []

    # Process conversation data (main training source)
    for item in conversation_data:
        conversations = item['conversations']

        # Convert to Gemma 3n chat format
        formatted_text = ""
        for conv in conversations:
            role = conv['role']
            content = conv['content']

            if role == 'user':
                formatted_text += f"<start_of_turn>user\n{content}<end_of_turn>\n"
            elif role == 'assistant':
                formatted_text += f"<start_of_turn>model\n{content}<end_of_turn>\n"

        training_texts.append({"text": formatted_text})

    # Add speech patterns for better understanding
    for item in speech_data:
        conversations = item['conversations']
        if len(conversations) >= 2:
            assistant_content = conversations[1]['content']

            # Create speech understanding sample
            formatted_text = f"<start_of_turn>user\nMi hermana dice: {assistant_content}<end_of_turn>\n"
            formatted_text += f"<start_of_turn>model\n¡Perfecto! Entendí: '{assistant_content}'. ¿En qué te puedo ayudar? 😊<end_of_turn>\n"

            training_texts.append({"text": formatted_text})

    print(f" Total training samples: {len(training_texts)}")
    return training_texts

# Load and format data
speech_data, conversation_data = load_training_data()
training_texts = format_for_training(speech_data, conversation_data)

print(f"\n Training on {len(training_texts)} samples sister's voice!")

 Loading sister's training data...
 Speech samples: 202
 Conversation samples: 202
 Formatting data for training...
 Total training samples: 404

 Training on 404 samples sister's voice!


Initialize Gemma3n Model

In [ ]:
from unsloth import FastLanguageModel

print(" Loading Gemma 3n model with Unsloth...")

# Load Gemma model (using 2B for faster training)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Gemma-2-2b-it",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

# Add LoRA adapters for efficient fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

print("Model loaded successfully!")
print("LoRA adapters added for efficient training!")
print("Ready to train users's personalized assistant!")

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.2.2+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.8 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
 Loading Gemma 3n model with Unsloth...
==((====))==  Unsloth 2025.7.11: Fast Gemma2 patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Unsloth 2025.7.11 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


Model loaded successfully!
LoRA adapters added for efficient training!
Ready to train users's personalized assistant!


Setup Training

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Create dataset
dataset = Dataset.from_list(training_texts)
print(f" Dataset created with {len(dataset)} samples")

# Training arguments optimized for sister's voice
training_args = TrainingArguments(
    output_dir="./sister_dream_assistant",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=150,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=3,
    report_to="none",
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=training_args,
)

print("Training setup complete!")
print("Ready to create users's personalized Dream Assistant!")

 Dataset created with 404 samples


Unsloth: Tokenizing ["text"]:   0%|          | 0/404 [00:00<?, ? examples/s]

Training setup complete!
Ready to create users's personalized Dream Assistant!


Training

In [ ]:
print("Starting fine-tuning for Sister's Dream Assistant...")
print("Creating a personalized model that understands her unique speech patterns!")
print("Training it to be her best friend and business mentor!")
print("\nThis will take about 15-20 minutes...")

# Start training
trainer.train()

print("\n TRAINING COMPLETE!")
print("You now have your own personalized Gemma 3n model!")

Starting fine-tuning for Sister's Dream Assistant...
Creating a personalized model that understands her unique speech patterns!
Training it to be her best friend and business mentor!

This will take about 15-20 minutes...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 404 | Num Epochs = 3 | Total steps = 150
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 20,766,720 of 2,635,108,608 (0.79% trained)


Step,Training Loss
1,7.638700
2,7.287900
3,6.687600
4,6.062300
5,5.476300
6,4.081800
7,3.555600
8,2.825400
9,2.663300
10,2.417800



 TRAINING COMPLETE!
You now have your own personalized Gemma 3n model!


Testing the model

In [ ]:
print(" Testing users's personalized Dream Assistant...")

# Enable inference mode
FastLanguageModel.for_inference(model)

# Testing with realistic imputs
test_inputs = [
    "Hola, ¿cómo estás?",
    "Ayúdame con mi negocio de la plataforma",
    "Quiero enviar un mensaje a mi mamá que diga hola",
    "Me siento un poco triste hoy",
    "Abre YouTube para grabar un video sobre mi proyecto",
    "Necesito crear una reunión para mi comunidad",
    "No sé si puedo lograr mi sueño de emprender"
]

for test_input in test_inputs:
    inputs = tokenizer(
        f"<start_of_turn>user\n{test_input}<end_of_turn>\n<start_of_turn>model\n",
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.7,
        do_sample=True,
        use_cache=True
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.split("<start_of_turn>model\n")[-1]

    print(f" Tu hermana: {test_input}")
    print(f" Dream Assistant: {response}")
    print("-" * 60)

print("\n Personalized and supportive responses")
print(" Perfect for users's needs!")

 Testing users's personalized Dream Assistant...
 Tu hermana: Hola, ¿cómo estás?
 Dream Assistant: user
Hola, ¿cómo estás?
model
¡Hola! Me da mucha alegría escucharte. ¿Cómo estás hoy?
------------------------------------------------------------
 Tu hermana: Ayúdame con mi negocio de la plataforma
 Dream Assistant: user
Ayúdame con mi negocio de la plataforma
model
¡Me encanta tu espíritu emprendedor! Estoy aquí para apoyarte en todos tus proyectos de negocio. Cuéntame más sobre tu idea y te ayudo a desarrollarla.
------------------------------------------------------------
 Tu hermana: Quiero enviar un mensaje a mi mamá que diga hola
 Dream Assistant: user
Quiero enviar un mensaje a mi mamá que diga hola
model
Te escucho perfectamente. Estoy aquí para ayudarte en lo que necesites.
------------------------------------------------------------
 Tu hermana: Me siento un poco triste hoy
 Dream Assistant: user
Me siento un poco triste hoy
model
Te entiendo perfectamente. Estoy aquí para esc

Saving for Android

In [ ]:
# Enable inference mode first
FastLanguageModel.for_inference(model)

print(" Preparing model for Android deployment...")

# Save the fine-tuned model (LoRA adapters)
model.save_pretrained("sister_dream_assistant_lora")
tokenizer.save_pretrained("sister_dream_assistant_lora")

print(" LoRA model saved!")

# Convert to GGUF format for Android
print(" Converting to mobile-optimized GGUF format...")
print("This will take a few minutes...")

model.save_pretrained_gguf(
    "sister_dream_assistant_gguf",
    tokenizer,
    quantization_method="q4_k_m"  # Good balance of size and quality
)

print("Model saved in multiple formats!")

# Check what was created
import os
if os.path.exists("sister_dream_assistant_gguf"):
    print(" GGUF files created:")
    total_size = 0
    for file in os.listdir("sister_dream_assistant_gguf"):
        file_path = f"sister_dream_assistant_gguf/{file}"
        size_mb = os.path.getsize(file_path) / (1024*1024)
        total_size += size_mb
        print(f"  {file}: {size_mb:.1f} MB")
    print(f" Total GGUF size: {total_size:.1f} MB")

# NOW save to Google Drive
print("\n Now saving to Google Drive...")

# Drive should already be mounted from before
drive_folder = "/content/drive/MyDrive/DreamAssistant_Models"
os.makedirs(drive_folder, exist_ok=True)

import shutil

# Copy the main quantized model file (this is what we need for Android)
main_model = "sister_dream_assistant_gguf/unsloth.Q4_K_M.gguf"
if os.path.exists(main_model):
    model_size_gb = os.path.getsize(main_model) / (1024*1024*1024)
    print(f" Copying {model_size_gb:.2f}GB model to Google Drive...")
    print(" This will take a few minutes but is more reliable than direct download...")

    shutil.copy2(main_model, f"{drive_folder}/sister_dream_assistant.gguf")

    # Copy essential config files
    essential_files = [
        "sister_dream_assistant_gguf/config.json",
        "sister_dream_assistant_gguf/generation_config.json",
        "sister_dream_assistant_gguf/tokenizer.json",
        "sister_dream_assistant_gguf/tokenizer_config.json",
        "sister_dream_assistant_gguf/tokenizer.model"
    ]

    for config_file in essential_files:
        if os.path.exists(config_file):
            filename = os.path.basename(config_file)
            shutil.copy2(config_file, f"{drive_folder}/{filename}")
            print(f"📄 Copied {filename}")

    # Create deployment info
    import json
    deployment_info = {
        "model_info": {
            "name": "Sister's Dream Assistant",
            "base_model": "Gemma-2-2b-it",
            "fine_tuned_on": "202+ voice samples from sister",
            "specialization": "Speech impairment + motivational responses",
            "file_size_gb": round(model_size_gb, 2),
            "format": "GGUF Q4_K_M (mobile optimized)",
            "training_quality": "Excellent (loss: 7.638 → 0.190)"
        },
        "android_deployment": {
            "main_file": "sister_dream_assistant.gguf",
            "required_files": ["config.json", "tokenizer files"],
            "integration_ready": True,
            "privacy": "100% offline and private"
        },
        "hackathon": {
            "competition": "Google Gemma 3n Impact Challenge",
            "prizes_eligible": ["Unsloth Prize", "Google AI Edge Prize", "Main Competition"],
            "impact": "Accessibility for speech impairment + community building"
        }
    }

    with open(f"{drive_folder}/deployment_info.json", "w") as f:
        json.dump(deployment_info, f, indent=2)

    print(" SUCCESS! Model saved to Google Drive!")
    print("=" * 50)
    print(" Location: MyDrive/DreamAssistant_Models/")
    print(" Files ready for Android:")
    print("   sister_dream_assistant.gguf (main model)")
    print("  config.json + tokenizer files")
    print("   deployment_info.json")



else:
    print(" GGUF model file not found after conversion")
    print("Let me check what was actually created...")
    if os.path.exists("sister_dream_assistant_gguf"):
        print("GGUF folder contents:", os.listdir("sister_dream_assistant_gguf"))

 Preparing model for Android deployment...


Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


 LoRA model saved!
 Converting to mobile-optimized GGUF format...
This will take a few minutes...


Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.2G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 32.11 out of 50.99 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


...

Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting gemma2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at sister_dream_assistant_gguf into f16 GGUF format.
The output location will be /content/sister_dream_assistant_gguf/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: sister_dream_assistant_gguf
INFO:hf-to-gguf:Model architecture: Gemma2ForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /content/sister_dream_assistant_gguf/unsloth.Q4_K_M.gguf
Model saved in multiple formats!
 GGUF files created:
  model.safetensors.index.json: 0.0 MB
  tokenizer.model: 4.0 MB
  generation_config.json: 0.0 MB
  unsloth.F16.gguf: 4992.7 MB
  unsloth.Q4_K_M.gguf: 1629.4 MB
  special_tokens_map.json: 0.0 MB
  tokenizer.json: 32.8 MB
  model-00002-of-00002.safetensors: 229.5 MB
  config.json: 0.0 MB
  tokenizer_config.json: 0.0 MB
  model-00001-of-00002.safetensors: 4757.0 MB
  chat_template.jinja: 0.0 MB
 Total GGUF size: 11645.5 MB

 Now saving to Google Drive...
 Copying 1.59GB model to Google Drive...
 This will take a few minutes but is more reliable than direct download...
📄 Copied config.json
📄 Copied generation_config.json
📄 Copied tokenizer.json
📄 Copied tokenizer_config.json
📄 Copied tokenizer.model
 SUCCESS! Model saved to Google Drive!
 Location: MyDrive/DreamAssistant_Models/
 Files ready for Android:
   sister_dream_assistant.gg